<a href="https://colab.research.google.com/github/Nancytechie/Data-Science-/blob/main/AI_Lab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install advertools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.1/312.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.4/286.4 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.4/97.4 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 7.9 MB/s eta 0:00:00


In [ ]:
pip install ua-parser

In [ ]:
pip install squarify

In [ ]:
import advertools as adv
import pandas as pd
from ua_parser import user_agent_parser
import pyarrow.parquet as pq
import pyarrow
from ipywidgets import interact
import matplotlib.pyplot as plt
import ua_parser
pd.options.display.max_columns = None
import squarify

In [5]:
%%time
adv.logs_to_df(
    log_file='/content/log_file.csv',
    output_file='output_file.parquet',
    errors_file='errors_file.txt',
    log_format='combined')

Parsed           6,001 lines.
CPU times: user 124 ms, sys: 30.9 ms, total: 155 ms
Wall time: 352 ms


In [6]:
logs_df = pd.read_parquet('output_file.parquet')
logs_df['datetime'] = pd.to_datetime(logs_df['datetime'],
                                     format='%d/%b/%Y:%H:%M:%S %z')

In [7]:
from collections import defaultdict
import re

# Create a dictionary to store log segments for each hour
log_segments = defaultdict(list)

# Regular expression to match the timestamp
timestamp_pattern = r'\[([^:]+):\d{2}:\d{2}:\d{2}'

for log_entry in logs_df:
    # Extract the timestamp using regex
    match = re.search(timestamp_pattern, log_entry)
    if match:
        timestamp = match.group(1)

        # Extract the hour from the timestamp
        hour = timestamp.split(':')[1]

        # Append the log entry to the corresponding hour's segment
        log_segments[hour].append(log_entry)

# Print the log segments for each hour
for hour, segment in log_segments.items():
    print(f"Hour: {hour}")
    for entry in segment:
        print(entry)
    print("\n")

In [8]:
from ast import parse
import os
from flask import Flask, render_template, request, send_file

In [9]:
app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload():
    if 'log_file' not in request.files:
        return "No file part"

    log_file = request.files['log_file']

    if log_file.filename == '':
        return "No selected file"
    if log_file:

        file_contents = log_file.read()
        with open('logfile.log', 'w') as log:
            log.write(file_contents.decode('utf-8'))
        adv.logs_to_df(
            log_file='logfile.log',
            output_file='output_file.parquet',
            errors_file='errors_file.txt',
            log_format='combined'
        )
        logs_df = pd.read_parquet('output_file.parquet')
        logs_df['datetime'] = pd.to_datetime(logs_df['datetime'],
                                    format='%d/%b/%Y:%H:%M:%S %z')
        report = generate_report(logs_df)
        report_filename = 'report.txt'
        with open(report_filename, 'w') as report_file:
            report_file.write(report)
        report = generate_report(logs_df)
    return send_file(report_filename, as_attachment=True, download_name='report.txt')




In [10]:
def generate_report(logs_df):
    # Example: Calculate the total number of log entries
    total_entries = len(logs_df)

    # Example: Calculate the unique IP addresses
    unique_ips = logs_df['client'].nunique()

    # Example: Calculate the number of HTTP status codes
    status_code_counts = logs_df['status'].value_counts()
    summary_stats=logs_df.describe()
    info=logs_df.info()
    inf=dataframe_to_text(info)
    # Generate summary statistics for numeric columns
    #numeric_summary = logs_df.describe(include='number')
    ns=dataframe_to_text(summary_stats)
    top_visitors = dataframe_to_text(logs_df['client'].value_counts().head(10))
    frequency_of_requests = dataframe_to_text(logs_df['request'].value_counts().head(10))
    status_df = dataframe_to_text(logs_df.groupby('status').size().reset_index(name='count'))
    logs_df['hour'] = logs_df['datetime'].dt.hour
    logs_df['day_of_week'] = logs_df['datetime'].dt.dayofweek
    # Group by IP address and hour, and calculate the count of requests
    ip_hourly_counts = dataframe_to_text(logs_df.groupby(['client', 'hour'])['hour'].count().reset_index(name='request_count').head(5))
    # Group by IP address and day of the week, and calculate the average request count
    ip_weekly_avg = dataframe_to_text(logs_df.groupby(['client', 'day_of_week'])['day_of_week'].count().reset_index(name='request_count').head(5))
    logs_df['browser'] = logs_df['user_agent'].apply(extract_browser)
    browser_hits = dataframe_to_text(logs_df.groupby('browser').size().reset_index(name='total_hits'))
    browser_os_df = ua_df[['ua_family', 'ua_os.family']]
    # Count the number of occurrences of each browser and operating system combination
    browser_os_counts = browser_os_df.groupby(['ua_family', 'ua_os.family']).size().reset_index(name='count')
    # Example: Generate a simple report
    report = f"Log Analysis Report\n\n"
    report += f"Total Log Entries: {total_entries}\n"
    report += f"Unique IP Addresses: {unique_ips}\n"
    report += "HTTP Status Code Counts:\n"
    for code, count in status_code_counts.items():
        report += f"  - {code}: {count}\n"
    report += "\nSummary Statistics (Numeric Columns):\n"
    report+=ns
    report += "\nDataFrame Information:\n"
    report+=inf
    report+="\nTop Visitors:\n"
    report+=top_visitors
    report+="\nFrequency:\n"
    report+=frequency_of_requests
    report+="\nStatus:\n"
    report+=status_df
    report+="\nIP_Hourly_count:\n"
    report+=ip_hourly_counts
    report+="\nIP Weekly Count:\n"
    report+=ip_weekly_avg
    report+="\nBrowser Hits\n"
    report+=browser_hits
    return report

In [11]:

def dataframe_to_text(df):
    if df is None:
        return "No data available for report"
    return df.to_string(index=False)


In [12]:
def extract_browser(user_agent):
    ua = parse(user_agent)
    browser_family = ua.browser.family.lower()

    if 'chrome' in browser_family:
        return 'Chrome'
    elif 'firefox' in browser_family:
        return 'Firefox'
    elif 'safari' in browser_family:
        return 'Safari'
    elif 'edge' in browser_family:
        return 'Edge'
    elif 'opera' in browser_family:
        return 'Opera'
    elif 'brave' in browser_family:
        return 'Brave'
    elif 'internet explorer' in browser_family or 'msie' in browser_family:
        return 'Internet Explorer'
    else:
        return 'Other'
if __name__ == '_main_':
    app.run(debug=True)
